In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import importlib

import my_utils.pixel as pixel
importlib.reload(pixel) # get changes in my_utils.pixel
print(os.getcwd())

/home/lukas/Documents/ETH/MASTER_THESIS/code


### Load Data:

In [2]:
WW = True  # only consider 'Winter Wheat' otherwise consider Cereals

if WW:
    path_cov = os.path.join("data/yieldmapping_data", "WW_covariates_tot.csv")
    path_met = os.path.join("data/yieldmapping_data", "WW_meteo_tot.csv")
    path_yie = os.path.join("data/yieldmapping_data", "WW_yield_tot.csv")
else:
    path_cov = os.path.join("data/yieldmapping_data",
                            "Cereals_covariates_tot.csv")
    path_met = os.path.join("data/yieldmapping_data", "Cereals_meteo_tot.csv")
    path_yie = os.path.join("data/yieldmapping_data", "Cereals_yield_tot.csv")

d_cov = pd.read_csv(path_cov)
d_met = pd.read_csv(path_met)
d_yie = pd.read_csv(path_yie)

In [6]:
from sklearn.model_selection import KFold
import random 

np.random.seed(1)
self = pixel.Pixel(d_cov, d_met, d_yie)

def interpolation_cv(self, methodname, y=None, k=5, n_min=5, method='get_smooting_spline', kwargs={}):
    """
    teile irgendwie crossvalidation ein,
    gewünscht: k is die anzahl der durchgänge
    n_min ist die mindeste geforderte anzahl an punkten zum training

    result:
    DataFrame where each column corresponds to the error of one test-fold

    from now on apply furtcher functions which calculate statistics
    like RMSE, Med, MAD, QN, ....
    """
    if k is None:
        k = self.cov_n
    if y is None:
        y = self.get_ndvi()   
    k = min(self.cov_n-2, k)
    kf = KFold(k, shuffle=True)
    ## alternative way which might be used for keeping start and end
    # def partition (list_in, n):
        # random.shuffle(list_in)
        # return [list_in[i::n] for i in range(n)]
    create a new dataframe
    for i, train, test in enumerate(interpolation_cv(self)):
        ind_keep = 
        name = "cv_"+methodname+"_"+str(i)
        args = {**kwargs, 'ind_keep':ind_keep, 'name':name}
        getattr(self, method)(**args)
        ~compare estimation with truth for test
        ~get residuals in data frame

        
    return kf.split(y)

interpolation_cv(self, kwargs={'name':"WW"})
self.step_interpolate

# self.get_smooting_spline(smooth=1e-11/(28*24*3600), name="ss")
# self.plot_ndvi()
# self.plot_step_interpolate(which="ss")



# for train, test in interpolation_cv(self):
#     print("%s %s" % (train, test))
#     for i in test:
#         print(self.cov.date.iloc[i])

,date,date_unix,WW
0,2019-02-24,1.550966e+09,0.352364
1,2019-02-25,1.551053e+09,0.369764
2,2019-02-26,1.551139e+09,0.387037
3,2019-02-27,1.551226e+09,0.404055
4,2019-02-28,1.551312e+09,0.420691
...,...,...,...
140,2019-07-14,1.563062e+09,0.313605
141,2019-07-15,1.563149e+09,0.301976
142,2019-07-16,1.563235e+09,0.288417
143,2019-07-17,1.563322e+09,0.273411


### smoothing splines for NDVI:

In [ ]:
np.random.seed(1)
smooth = 0 #1e-11/(28*24*3600)

for i in range(2):
    temp = pixel.Pixel(d_cov, d_met, d_yie)
    temp.get_ndvi()
    temp.plot_ndvi()
    temp.get_smooting_spline(name="all", smooth=smooth)
    temp.get_smooting_spline(name="filter", smooth=smooth,
                             ind_keep=temp.filter("ndvi_min"))
    temp.plot_step_interpolate(which="all")
    temp.plot_step_interpolate(which="filter")
    plt.show()

# print(temp)

In [ ]:
import scipy.interpolate as interpolate
self = pixel.Pixel(d_cov, d_met, d_yie)
self.get_b_spline()
self.plot_ndvi()
self.plot_step_interpolate("BSpline")

# TO DO 
- beautiful plot (colors, legend, title)
- advanced plotting (support for lists)